In [1]:
import numpy as np 
import pandas as pd 
import json
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import glob
from skimage.feature import hog
from PIL import Image
import os.path, sys
from skimage.io import imread, imshow
import cv2
from skimage import data, color, feature , exposure
import imgaug.augmenters as iaa
from skimage.transform import resize
from skimage.feature import hog
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns; sns.set()

from _helper_functions import *

Using TensorFlow backend.


# Extract HOG Features - Train Images

In [8]:
path = '/Users/toothless/Downloads/Thesis/data/Train/original/_nocancer/'

In [9]:
re_ncan = resize_aspect_fit(path,150)

In [10]:
nchog_feat = []
nchog_img = []
for img in re_ncan:
    #image = mpimg.imread(img)
    fd,hog_image = hog(img, orientations=8, pixels_per_cell=(10,10),
                       cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    nchog_feat.append(fd)
    nchog_img.append(hog_image)
    

In [11]:
nchog_img = np.array(nchog_img)

In [12]:
nchog_feat = np.array(nchog_feat)

In [13]:
print(nchog_img.shape)
print(nchog_feat.shape)

(6778, 150, 150)
(6778, 18432)


In [14]:
path = '/Users/toothless/Downloads/Thesis/data/Train/original/cancer/'
re_can = resize_aspect_fit(path,150)

In [15]:
chog_feat = []
chog_img = []
for img in re_can:
    #image = mpimg.imread(img)
    fd,hog_image = hog(img, orientations=8, pixels_per_cell=(10,10),
                       cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    chog_feat.append(fd)
    chog_img.append(hog_image)

In [16]:
chog_img = np.array(chog_img)
chog_feat = np.array(chog_feat)

In [17]:
print(chog_img.shape)
print(chog_feat.shape)

(7272, 150, 150)
(7272, 18432)


In [18]:
can_labels = np.ones(chog_img.shape[0])
nocan_labels = np.zeros(nchog_img.shape[0])

hog_feat = np.concatenate((chog_feat,nchog_feat),axis = 0)
hog_labels = np.concatenate((can_labels,nocan_labels) , axis = 0)
hog_images = np.concatenate((chog_img,nchog_img),axis = 0)

In [19]:
print(hog_feat.shape)
print(hog_labels.shape)
print(hog_images.shape)

(14050, 18432)
(14050,)
(14050, 150, 150)


In [37]:
np.savez_compressed('/Users/toothless/Downloads/Thesis/data/feat/Train_hog',
                    hog_feat=hog_feat, hog_img=hog_images,hog_labels=hog_labels)

# Extract HOG Features - Test Images

In [2]:
test_data = pd.read_csv('/Users/toothless/Downloads/Thesis/data/test_labels/test_data_labels.csv')

filename = test_data['new_names'].values

path = '/Users/toothless/Downloads/Thesis/data/Train_Test/testoriginal/'

In [3]:
test_images = []
for file in filename:
    test_images.append(path+file)

In [4]:
test_re = []
final_size=150
for img in test_images:
    im = Image.open(img)
    f, e = os.path.splitext(img)
    size = im.size
    ratio = float(final_size) / max(size)
    new_image_size = tuple([int(x*ratio) for x in size])
    im = im.resize(new_image_size, Image.ANTIALIAS)
    new_im = Image.new("RGB", (final_size, final_size))
    new_im.paste(im, ((final_size-new_image_size[0])//2, (final_size-new_image_size[1])//2))
    test_re.append(np.asarray(new_im))
            

In [5]:
test_re = np.array(test_re)

In [6]:
test_feat = []
test_img = []
for img in test_re:
    #image = mpimg.imread(img)
    fd,hog_image = hog(img, orientations=8, pixels_per_cell=(10,10),
                       cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    test_feat.append(fd)
    test_img.append(hog_image)

In [7]:
test_hog = np.array(test_feat)
test_img = np.array(test_img)
test_labels = np.array(test_data['labels'])

In [8]:
np.savez_compressed('/Users/toothless/Downloads/Thesis/data/feat/Test_hog',
                    testhog_feat=test_hog, testhog_img=test_img, labels=test_labels)